#Exercise for multiclass classification

#Check for GPU

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-321e0692-9620-44b9-7b69-13513cba4bef)


#Imports

In [4]:
import os

import tensorflow as tf

#Get dataset

In [12]:
path = tf.keras.utils.get_file('stack-overflow-questions',
                  'https://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz',
                  untar=True)

#Walkthrough dataset folders


In [27]:
dataset_dir = os.path.dirname(path)
os.listdir(dataset_dir)



['test', 'train', 'README.md', 'stack-overflow-questions.tar.gz']

In [28]:
train_dir = os.path.join(datset_dir, 'train')

os.listdir(train_dir)

['javascript', 'python', 'csharp', 'java']

In [30]:
test_dir = os.path.join(datset_dir, 'test')

#Use text_dataset_from_directory

In [33]:
seed=42

raw_train_ds = tf.keras.utils.text_dataset_from_directory(train_dir,
                                                          seed=seed,
                                                          validation_split=0.2,
                                                          subset='training')

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [34]:
val_ds = tf.keras.utils.text_dataset_from_directory(train_dir,
                                                          seed=seed,
                                                          validation_split=0.2,
                                                          subset='validation')

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [35]:
test_ds = tf.keras.utils.text_dataset_from_directory(test_dir)

Found 8000 files belonging to 4 classes.


#Look Through Dataset

In [39]:
for text, label in raw_train_ds.take(1):
  for i in range(3):
      print(text.numpy()[i])
      print(label.numpy()[i])

b'"blank8 why is my solution faster than the neat solution? (hackerrank chocolate feast) edit: simplified my solution..edit: removed opinion based secondary question...background: atarted learning blank a week or two ago using hackerranks problems as exercises and stackoverflow search + google as my teacher, i\'ve had some limited experience learning other languages...i did the exercise my own ""noobish learner way"" which i can\'t help but feel is a ""botched job"" when i see ""neat &amp; short"" solutions...however, when submitting both solutions one after another a couple of times i found the ""neat"" solution was quite a bit slower. ..i vaguely remember something about % operations being costly, is mine faster because of no % operations or is there more to it than just that?..exercise: https://www.hackerrank.com/challenges/chocolate-feast..neat solution from discussion:..import blank.io.*;.import blank.util.*;..public class solution {.    static int cc; .    public static void main

#Text vectorization

In [44]:
text_vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=10000,
    output_sequence_length=250
)

train_text = raw_train_ds.map(lambda x, y: x)

text_vectorizer.adapt(train_text)

#Perf

In [45]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


#Create the model

#Create embedding layer

In [47]:
embeddings = tf.keras.layers.Embedding(input_dim=10000+1,output_dim=16)

In [71]:
from tensorflow.keras import layers

num_classes=4

# inputs = layers.Input(shape=(1,))
# x=text_vectorizer(inputs)
# x=embeddings(x)
# x=layers.Dropout(0.2)(x)
# x=layers.GlobalAveragePooling1D()(x)
# x=layers.Dropout(0.2)(x)
# outputs=layers.Dense(num_classes, activation='softmax')(x)
# model=tf.keras.Model(inputs, outputs)

model= tf.keras.Sequential([
    text_vectorizer,
    embeddings,
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='Adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy, 
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [72]:
model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10


TypeError: ignored

#Evaluate the model

#Plot loss curves
